# Imports + Git

In [ ]:
# prompt: ERROR: file:///content/TTS/TTS/tts/layers/xtts does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found

!git clone https://github.com/coqui-ai/TTS
%cd TTS
!pip install -e .[all,dev,notebooks]  # Select the relevant extras

In [ ]:
!pip install numpy==1.23

Testing tokonizer

In [ ]:
from TTS.TTS.tts.layers.xtts.tokenizer import VoiceBpeTokenizer

# Path to the vocab.json file
vocab_path = "/content/vocab.json"

# Initialize the tokenizer
tokenizer = VoiceBpeTokenizer(vocab_path)


In [ ]:
# Text to tokenize
text = "صباح الخير"

# Tokenize the text (encode it)
tokens = tokenizer.encode(text, lang="ar")

# Print tokenized output
print("Tokenized Output:", tokens)


Tokenized Output: [5022, 3961, 4066, 3953, 2, 4268, 4105]


# Importing Dataset from Google drive

downloading dataset from google drive

In [ ]:
%cd /content

/content


In [ ]:
!gdown --id 1-EB55BAs92xLQa_sM3PptWZ6xlJ6yZhY

In [ ]:
import zipfile
import os

# Create the target directory
target_dir = '/content/lJspeech'
os.makedirs(target_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile("/content/txt_to_speech_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall(target_dir)

print(f'File successfully unzipped to {target_dir}')


File successfully unzipped to /content/lJspeech


# Model Config + Training

In [ ]:
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager

# Logging parameters
RUN_NAME = "GPT_XTTS_v2.0_LJSpeech_FT"
PROJECT_NAME = "XTTS_trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

# Set here the path that the checkpoints will be saved. Default: ./run/training/
OUT_PATH = "/content/run/training/"

# Training Parameters
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True  # for multi-gpu training please make it False
START_WITH_EVAL = True  # if True it will star with evaluation
BATCH_SIZE = 3  # set here the batch size
GRAD_ACUMM_STEPS = 84  # set here the grad accumulation steps
# Note: we recommend that BATCH_SIZE * GRAD_ACUMM_STEPS need to be at least 252 for more efficient training. You can increase/decrease BATCH_SIZE but then set GRAD_ACUMM_STEPS accordingly.

# Define here the dataset that you want to use for the fine-tuning on.
config_dataset = BaseDatasetConfig(
    formatter="ljspeech",
    dataset_name="ljspeech",
    path="/content/lJspeech/",
    meta_file_train="/content/lJspeech/metadata.csv",
    language="ar",
)

# Add here the configs of the datasets
DATASETS_CONFIG_LIST = [config_dataset]


In [ ]:

# Define the path where XTTS v2.0.1 files will be downloaded
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)


# DVAE files
DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

# download DVAE files if needed
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=False)


# Download XTTS v2.0 checkpoint if needed
TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

# XTTS transfer learning parameters: You we need to provide the paths of XTTS model checkpoint that you want to do the fine tuning.
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json file
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth file

# download XTTS v2.0 files if needed
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=False
    )


# Training sentences generations
SPEAKER_REFERENCE = [
    "/content/lJspeech/wavs/audio_10302.wav"  # speaker reference to be used in training test sentences
]
LANGUAGE = config_dataset.language




In [ ]:
def main():
    # init args and config
    model_args = GPTArgs(
        max_conditioning_length=132300,  # 6 secs
        min_conditioning_length=66150,  # 3 secs
        debug_loading_failures=False,
        max_wav_length=255995,  # ~11.6 seconds
        max_text_length=200,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,  # checkpoint path of the model that you want to fine-tune
        tokenizer_file=TOKENIZER_FILE,
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True,
    )
    # define audio config
    audio_config = XttsAudioConfig(sample_rate=44100, dvae_sample_rate=22050, output_sample_rate=24000)
    # training parameters config
    config = GPTTrainerConfig(
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="""
            GPT XTTS training
            """,
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=48,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=8,
        eval_split_max_size=256,
        print_step=50,
        plot_step=100,
        log_model_step=1000,
        save_step=10000,
        save_n_checkpoints=1,
        save_checkpoints=True,
        # target_loss="loss",
        print_eval=False,
        # Optimizer values like tortoise, pytorch implementation with modifications to not apply WD to non-weight parameters.
        optimizer="AdamW",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=5e-06,  # learning rate
        lr_scheduler="MultiStepLR",
        # it was adjusted accordly for the new step scheme
        lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
        test_sentences=[
            {
                "text": "صباح الخير انا بتكلم مصرى عادى",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "اهلا انا محمود و دا المشروع",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
        ],
    )

    # init the model from config
    model = GPTTrainer.init_from_config(config)

    # load training samples
    train_samples, eval_samples = load_tts_samples(
        DATASETS_CONFIG_LIST,
        eval_split=True,
        eval_split_max_size=config.eval_split_max_size,
        eval_split_size=config.eval_split_size,
    )

    # init the trainer and 🚀
    trainer = Trainer(
        TrainerArgs(
            restore_path=None,  # xtts checkpoint is restored via xtts_checkpoint key so no need of restore it using Trainer restore_path parameter
            skip_train_epoch=False,
            start_with_eval=START_WITH_EVAL,
            grad_accum_steps=GRAD_ACUMM_STEPS,
        ),
        config,
        output_path=OUT_PATH,
        model=model,
        train_samples=train_samples,
        eval_samples=eval_samples,
    )
    trainer.fit()


if __name__ == "__main__":
    main()

# Uploading Training weights to drive

In [ ]:
# prompt: save this file in google drive
from google.colab import drive
drive.mount('/content/drive')

drive.mount("/content/drive", force_remount=True)
!mkdir -p /content/drive/MyDrive/weights/
#!cp -r /content/txt_to_speech_dataset /content/drive/MyDrive/txt_to_speech_dataset_files
!cp /content/run/training/GPT_XTTS_v2.0_LJSpeech_FT-January-05-2025_06+50AM-0000000/best_model.pth /content/drive/MyDrive/weights/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted at /content/drive


#Model Inference

In [ ]:
%cd /content/Finetuned_model
!gdown --id 1-131EwOwmJtgC5tg0B9CiGOpN0f1yOGz #checkpoint.pth
!gdown --id 1I38OqwNPx7ikwi-MoRhPjklUBJS-Q20d #config.json

Original Model

In [ ]:
import os
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

# Add here the xtts_config path
CONFIG_PATH = "/content/run/training/GPT_XTTS_v2.0_LJSpeech_FT-January-05-2025_06+50AM-0000000/config.json"
# Add here the vocab file that you have used to train the model
TOKENIZER_PATH = "/content/run/training/XTTS_v2.0_original_model_files/vocab.json"
# Add here the checkpoint that you want to do inference with
XTTS_CHECKPOINT = "/content/run/training/XTTS_v2.0_original_model_files/model.pth"
# Add here the speaker reference
SPEAKER_REFERENCE = "/content/lJspeech/wavs/audio_10302.wav"

# output wav path
OUTPUT_WAV_PATH = "Original.wav"

print("Loading model...")
config = XttsConfig()
config.load_json(CONFIG_PATH)
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_path=XTTS_CHECKPOINT, vocab_path=TOKENIZER_PATH, use_deepspeed=False)
model.cuda()

print("Computing speaker latents...")
gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path=[SPEAKER_REFERENCE])



Loading model...


/content/TTS/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


Computing speaker latents...
Inference...


Finetuned Model

In [ ]:
import os
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

# Add here the xtts_config path
CONFIG_PATH = "/content/run/training/GPT_XTTS_v2.0_LJSpeech_FT-January-05-2025_06+50AM-0000000/config.json"
# Add here the vocab file that you have used to train the model
TOKENIZER_PATH = "/content/run/training/XTTS_v2.0_original_model_files/vocab.json"
# Add here the checkpoint that you want to do inference with
XTTS_CHECKPOINT = "/content/run/training/GPT_XTTS_v2.0_LJSpeech_FT-January-05-2025_06+50AM-0000000/checkpoint_18347.pth"
# Add here the speaker reference
SPEAKER_REFERENCE = "/content/lJspeech/wavs/audio_10302.wav"

# output wav path
OUTPUT_WAV_PATH1 = "Finetuned.wav"

print("Loading model...")
config = XttsConfig()
config.load_json(CONFIG_PATH)
model1 = Xtts.init_from_config(config)
model1.load_checkpoint(config, checkpoint_path=XTTS_CHECKPOINT, vocab_path=TOKENIZER_PATH, use_deepspeed=False)
model1.cuda()

print("Computing speaker latents...")
gpt_cond_latent, speaker_embedding = model1.get_conditioning_latents(audio_path=[SPEAKER_REFERENCE])


Loading model...


/content/TTS/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


Computing speaker latents...
Inference...


In [ ]:
OUTPUT_WAV_PATH1 = "Finetuned.wav"


In [ ]:
OUTPUT_WAV_PATH = "Original.wav"

Testing FineTuned VS Original Model

In [ ]:
sentence =  "الصبح وأنا نازل من البيت لقيت الجو برد جدًا، فاضطريت أرجع أخد جاكيت، ولما وصلت للمحطة اكتشفت إن الميكروباصات كلها زحمة والناس واقفة مستنية"
#Test
gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path=[SPEAKER_REFERENCE])
out = model.inference(
   sentence,
    "ar",
    gpt_cond_latent,
    speaker_embedding,
    temperature=0.7, # Add custom parameters here
)
torchaudio.save(OUTPUT_WAV_PATH, torch.tensor(out["wav"]).unsqueeze(0), 24000)

out1 = model1.inference(
    sentence,
    "ar",
    gpt_cond_latent,
    speaker_embedding,
    temperature=0.7, # Add custom parameters here
)
torchaudio.save(OUTPUT_WAV_PATH1, torch.tensor(out1["wav"]).unsqueeze(0), 24000)

